In [1]:
'''
Haselnut - 0
Peanut - 0
Walnut - 0
'''

import torch.utils.data as data
import numpy as np
import glob
import os
import sys
import torch
import json
import platform
import csv
from IPython.display import display

from PIL import Image
if('/opt/ros/kinetic/lib/python2.7/dist-packages' in sys.path):
    sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
import cv2

def change_mask(mask, old_color, new_color):
    mask[np.where((mask==old_color).all(axis=2))] = new_color
    return mask

def mask_BW(image, mask):
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

def mask_transperent(mask):
    mask = change_mask(mask, [255,255,255], [0,255,0])
    return (255-mask)

def apply_overlay(image, mask, alpha):
    cv2.addWeighted(mask, alpha, image, 1 - alpha,0, image)
    return image

def get_polygon_mask(im, annotations):
    height,width,depth = im.shape
    mask = np.zeros(im.shape, np.uint8)
    ignore_mask_color = (255,)*depth
    for i,an in enumerate(annotations):
        polygon = an[1][np.newaxis]
        cv2.fillPoly(mask, polygon , ignore_mask_color)
        #u can remove below text later
#         cv2.putText(mask, an[0], (int(an[1][0,0]),int(an[1][0,1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2)
    return mask
    
def get_annotation(path):
    annotation = []
    with open(path) as f:
        objects = json.load(f)
    for obj in objects['shapes']:
        annotation.append([obj['label'] , np.array(obj['points'],dtype=np.uint)])
    return np.array(annotation)

def make_dataset(path):
    #default "../data/"
    data = []
    for root,d_names,f_names in os.walk(path):
        for f in f_names:
            if('.png' in f):
                data.append(os.path.join(root,f))
    return data

def read_bb_annotation(csv_file):  
    with open(csv_file) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        labels = []
        for row in readCSV:
            labels.append(row)
        return np.array(labels[1:])
    
def scale_image(im, bounding_boxes, wsize=416):
    w_old = im.shape[1]
    h_old = im.shape[0]
    w_new = wsize
    h_new = 0
    
    #find ratio of new height and width
    wpercent = (w_new/float(w_old))
    h_new = int((float(h_old)*float(wpercent)))
    hpercent = (h_new/float(h_old))
    
    #resize image
    resized_im = cv2.resize(im,(w_new,h_new))
    
    #resize bounding box
    new_bb = []
    for bb in bounding_boxes:
        x,y,w,h = int(bb[0]), int(bb[1]), int(bb[2]), int(bb[3])
        x = int(x*float(wpercent))
        y = int(y*float(hpercent))
        w = int((float(w)*float(wpercent)))
        h = int((float(h)*float(hpercent)))
        new_bb.append([x,y,w,h])
    return resized_im, np.array(new_bb,dtype=bounding_boxes.dtype)

class VisualTactile(data.Dataset):
    """
    Dataset to load images
    labels = {"Haselnut":0, "Peanut":1, "Walnut":2}
    """

    def __init__(self, datasetpath, transforms=None):
        self.path = datasetpath
        self.transforms = transforms
        self.images = make_dataset(self.path)
        self.annotation = [x.replace('image', 'label_renamed').replace(os.path.splitext(x)[-1], '.json') for x in self.images]
        self.videos = [x.replace('image', 'video').replace(os.path.splitext(x)[-1], '.avi') for x in self.images]
        self.labels = read_bb_annotation("/media/pritesh/Entertainment/cvData/bb_annotation.csv")
    def __getitem__(self,index):
        #read image and pre process as you like,
        #Get bounding box for particular image
        #Get label (0,1,2)
        #return these parameters
        pass

    def __len__(self):
        return len(self.data)

In [2]:
d = VisualTactile("/media/pritesh/Entertainment/cvData/")

In [ ]:
d.

In [3]:
#only to visualize data and recorrect any changes

# Upkey : 38
# DownKey : 40
# LeftKey : 37
# RightKey: 39
# Space : 32
# Delete : 3014656
# a : 97
# s : 115
# Esc : to quit window

#### Ubuntu 
# right : 83 
# left : 81

def write_title(img, path):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2

    cv2.putText(img,path, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    return img
def draw_BB(img, an_path, bb_data):
    rows = np.where(bb_data==an_path.split('/')[-1])[0]
    bb = bb_data[rows,:]
    for b in bb:
        x,y,w,h = int(b[1]), int(b[2]), int(b[3]), int(b[4])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    return img

def display_img(im_path, an_path):
    im = cv2.imread(data[count])
    annotation = get_annotation(an_path)
    mask = get_polygon_mask(im,annotation)
#     mask = mask_transperent(mask)
    masked_image = apply_overlay(im,mask, alpha = 0.5)
    
#     masked_image = apply_mask(im,mask)
#     mask = change_mask(mask, [255,255,255], [0,255,0])
#     masked_image = apply_overlay(im, (255-mask), alpha=0.5)
    
    write_title(masked_image, im_path)
    draw_BB(masked_image,an_path,bb_annotation)
    cv2.imshow("original", masked_image)
    cv2.imwrite("out/segmentation/{}".format(im_path.split("/")[-1]), masked_image)


# rows = np.where(ll==an_path.split('/')[-1])[0]
data = make_dataset("/media/pritesh/Entertainment/cvData/")
csv_file_path = "/media/pritesh/Entertainment/cvData/bb_annotation.csv"
labels = [x.replace('image', 'label_renamed').replace(os.path.splitext(x)[-1], '.json') for x in data]
bb_annotation = read_bb_annotation(csv_file_path)
count = 0
right = 83 if (platform.system() == "Linux") else 39
left = 81 if (platform.system() == "Linux") else 37
print("press right or left arrow key on the image window")

cv2.namedWindow("original", cv2.WINDOW_NORMAL)
while True:
    k = cv2.waitKey(100)
    if k == right:
        count +=1 
        if(count == len(data)):
            count=0
        display_img(data[count], labels[count])
    elif k == left:
        if(count == 0):
            count = len(data)-1
        else:
            count -= 1
        display_img(data[count], labels[count])
        continue
    elif k == 27:
        break
        
cv2.destroyAllWindows()

press right or left arrow key on the image window


In [ ]:
# Testing image mask
im = cv2.imread(data[2])
an = get_annotation(labels[2])
height,width,depth = im.shape
mask = np.zeros(im.shape, np.uint8)
ignore_mask_color = (255,)*depth
# polygon = np.array(an[0,1])[np.newaxis]
# cv2.fillPoly(mask, polygon , ignore_mask_color)

# polygon = np.array(an[1,1])[np.newaxis]
# cv2.fillPoly(mask, polygon , ignore_mask_color)
mask = get_polygon_mask(im, an)

# masked_image = cv2.bitwise_and(im, mask)
masked_image = apply_overlay(im, (255-mask), alpha=0.5)
# masked_image
cv2.namedWindow("original", cv2.WINDOW_NORMAL)
while True :
    cv2.imshow("original", masked_image)
    k = cv2.waitKey(100) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()

# Save resized and masked image for training

In [4]:
#only to visualize data and recorrect any changes

# Upkey : 38
# DownKey : 40
# LeftKey : 37
# RightKey: 39
# Space : 32
# Delete : 3014656
# a : 97
# s : 115
# Esc : to quit window

#### Ubuntu 
# right : 83 
# left : 81

def scale_image(im, bounding_boxes, wsize=416):
    w_old = im.shape[1]
    h_old = im.shape[0]
    w_new = wsize
    h_new = 0
    
    #find ratio of new height and width
    wpercent = (w_new/float(w_old))
    h_new = int((float(h_old)*float(wpercent)))
    hpercent = (h_new/float(h_old))
    
    #resize image
    resized_im = cv2.resize(im,(w_new,h_new))
    
    #resize bounding box
    new_bb = []
    for bb in bounding_boxes:
        x,y,w,h = int(bb[0]), int(bb[1]), int(bb[2]), int(bb[3])
        x = int(x*float(wpercent))
        y = int(y*float(hpercent))
        w = int((float(w)*float(wpercent)))
        h = int((float(h)*float(hpercent)))
        new_bb.append([x,y,w,h])
    return resized_im, np.array(new_bb,dtype=bounding_boxes.dtype)

def write_title(img, path):
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (10,30)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2

    cv2.putText(img,path, 
        bottomLeftCornerOfText, 
        font, 
        fontScale,
        fontColor,
        lineType)
    return img

def draw_BB(image, bounding_boxes):
    img = image.copy()
    for bb in bounding_boxes:
        x,y,w,h = int(bb[0]), int(bb[1]), int(bb[2]), int(bb[3])
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    return img

def write_file(path, bounding_box, label ,size):
    height,width = size[0], size[1]
    file1 = open(path+".txt","a") 
    # \n is placed to indicate EOL (End of Line)
    for i,bb in enumerate(bounding_box):
        x,y,w,h = float(bb[0])/float(width), float(bb[1])/float(height), \
            float(bb[2])/float(width), float(bb[3])/float(height)
        line = ["{} {:.5f} {:.5f} {:.5f} {:.5f}\n".format(label[i], x,y,w,h)]
        file1.writelines(line) 
    file1.close() #to change file access modes 
    
def display_img(im_path, an_path):
    im = cv2.imread(data[count])
    annotation = get_annotation(an_path)
    mask = get_polygon_mask(im,annotation)
#     mask = mask_transperent(mask)
#     masked_image = apply_overlay(im,mask, alpha = 0.5)
    masked_image = mask_BW(im, mask)
    rows = np.where(bb_annotation==an_path.split('/')[-1])[0]
    bb = bb_annotation[rows,:]
    resized_im, new_bb = scale_image(masked_image.copy(), bb[:,1:5])
    new_im = write_title(resized_im.copy(), an_path)
    new_im = draw_BB(new_im,new_bb)
    cv2.imshow("original", masked_image)
    cv2.imwrite("/media/pritesh/Entertainment/yolo/PyTorch-YOLOv3/data/custom/images/{}".format(im_path.split("/")[-1]), resized_im)
    write_file("/media/pritesh/Entertainment/yolo/PyTorch-YOLOv3/data/custom/labels/{}".format(im_path.split("/")[-1].split(".")[0]), new_bb, bb[:,-1], resized_im.shape)

# rows = np.where(ll==an_path.split('/')[-1])[0]
data = make_dataset("/media/pritesh/Entertainment/cvData/")
csv_file_path = "/media/pritesh/Entertainment/cvData/bb_annotation.csv"
labels = [x.replace('image', 'label_renamed').replace(os.path.splitext(x)[-1], '.json') for x in data]
bb_annotation = read_bb_annotation(csv_file_path)
count = 0
right = 83 if (platform.system() == "Linux") else 39
left = 81 if (platform.system() == "Linux") else 37
print("press right or left arrow key on the image window")

cv2.namedWindow("original", cv2.WINDOW_NORMAL)
while True:
    k = cv2.waitKey(100)
    if k == right:
        count +=1 
        if(count == len(data)):
            count=0
        display_img(data[count], labels[count])
    elif k == left:
        if(count == 0):
            count = len(data)-1
        else:
            count -= 1
        display_img(data[count], labels[count])
        continue
    elif k == 27:
        break
        
cv2.destroyAllWindows()



press right or left arrow key on the image window


In [12]:
#direct loop to write
def display_img(im_path, an_path, count):
    im = cv2.imread(im_path)
    annotation = get_annotation(an_path)
    mask = get_polygon_mask(im,annotation)
    masked_image = mask_BW(im.copy(), mask)
    
    rows = np.where(bb_annotation==an_path.split('/')[-1])[0]
    bb = bb_annotation[rows,:]
    resized_im, new_bb = scale_image(masked_image.copy(), bb[:,1:5])
    if((count+1)%5==0):
        resized_im = cv2.resize(im,(416, 261))
        cv2.imwrite("/media/pritesh/Entertainment/yolo/PyTorch-YOLOv3/data/samples/{}".format(im_path.split("/")[-1]), resized_im)
    else:
        cv2.imwrite("/media/pritesh/Entertainment/yolo/PyTorch-YOLOv3/data/custom/images/{}".format(im_path.split("/")[-1]), resized_im)
        write_file("/media/pritesh/Entertainment/yolo/PyTorch-YOLOv3/data/custom/labels/{}".format(im_path.split("/")[-1].split(".")[0]), new_bb, bb[:,-1], resized_im.shape)

    
data = make_dataset("/media/pritesh/Entertainment/cvData/")
csv_file_path = "/media/pritesh/Entertainment/cvData/bb_annotation.csv"
labels = [x.replace('image', 'label_renamed').replace(os.path.splitext(x)[-1], '.json') for x in data]
bb_annotation = read_bb_annotation(csv_file_path)
count = 0

for i,im_path in enumerate(data):
    display_img(im_path, labels[i], i)
    if(i==101):
        break

In [ ]:
labels[2].split("/")[-1].split(".")[0]

In [ ]:
np.where(bb_annotation=="CV19_label_renamed_100.json")

In [ ]:
cv2.resize(im,(416, 261))